In [3]:
from dotenv import load_dotenv
import logging
from rich.console import Console
from rich.prompt import Prompt

from stops_qa import (
    setup_database,
    load_data_to_db,
    create_sql_generator,
    execute_query,
    display_query_result,
    display_welcome,
    TEXTOS
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
console = Console()
load_dotenv()

True

In [4]:
try:
    conn = setup_database(db_name="stops.db")
    load_data_to_db(conn, csv_file="simulated_stops.csv")
    llm, prompt = create_sql_generator()
    display_welcome()

    while True:
        try:
            from IPython import get_ipython
            is_notebook = get_ipython() is not None
        except ImportError:
            is_notebook = False
            
        if is_notebook:
            question = input(TEXTOS["prompt_pregunta"])
        else:
            question = Prompt.ask(TEXTOS["prompt_pregunta"])
        
        if question.lower() == 'exit':
            console.print(TEXTOS["despedida"])
            break
            
        try:
            messages = prompt.format_messages(question=question)
            
            with console.status(TEXTOS["generando_sql"]) as status:
                response = llm.invoke(messages)
                sql_query = response.content.strip()
            
            with console.status(TEXTOS["ejecutando_query"]) as status:
                results, error = execute_query(conn, sql_query)
                display_query_result(sql_query, results, error)
            
        except Exception as e:
            logger.error(f"Error: {str(e)}")
            console.print(f"{TEXTOS['error']} {str(e)}")

except Exception as e:
    console.print(f"{TEXTOS['error_fatal']} {str(e)}")
finally:
    if 'conn' in locals():
        conn.close()

✓ Cargados 1422 registros en la base de datos.

╭─────────────────────────────────── Sistema de Análisis de Paradas de Máquina ───────────────────────────────────╮
│                                                                                                                 │
│ ¡Bienvenido a StopQuery!                                                                                        │
│                                                                                                                 │
│ Este sistema te permite hacer preguntas en lenguaje natural sobre las paradas                                   │
│ de máquina y las convierte en consultas SQL para su análisis.                                                   │
│                                                                                                                 │
│ Ejemplos de Preguntas:                                                                                          │
│ • ¿Cuántas paradas ocurrieron el 1 de enero?                                                                    │
│ • Lista todas las paradas causadas por 'Die Head Cleaning'                                                      │
│ • ¿Cuál es la duración promedio de cada tipo de parada?                                                         │
│ • ¿Qué hora del día tuvo la duración total de paradas más larga?                                                │
│                                                                                                                 │
│ Escribe 'exit' para salir                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:stops_qa:Ejecutando SQL: SELECT stop_type_id, stop_type_name, AVG(duration_minutes) AS avg_duration
FROM stops
GROUP BY stop_type_id, stop_type_name;


Consulta SQL Generada:

SELECT stop_type_id, stop_type_name, AVG(duration_minutes) AS avg_duration                                         
FROM stops                                                                                                         
GROUP BY stop_type_id, stop_type_name;                                                                             

Resultados:

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ stop_type_id                         ┃ stop_type_name              ┃ avg_duration       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ 010d32a2-0abe-481e-ae14-eda32648b432 │ Pressure Sensor Calibration │ 44.945454545454545 │
│ 059d662f-bcd2-4368-950e-60d3ac49898f │ Emergency Safety Stop       │ 58.15              │
│ 1ba11308-7a25-4ed9-aea4-daa02fe863e3 │ Cooling System Check        │ 31.035000000000004 │
│ 3c251b2a-6fa2-4707-ad22-25858d52ebce │ Screw Wear Inspection       │ 176.03333333333333 │
│ 43e0f14d-c79c-47b5-9402-ef93f6e407b0 │ Screen Pack Change          │ 55.502272727272725 │
│ 44c73884-5c8e-48a6-a4ed-ee920cddfd7f │ Drive Motor Overload        │ 205.5              │
│ 44d3afce-8bc3-4269-9c1b-f630aedb9668 │ Quality Sample Check        │ 9.301428571428572  │
│ 4ff8deef-be87-40bf-adb9-8f2e102c1f23 │ Gearbox Maintenance         │ 623.8              │
│ 77322f6a-0ed2-4946-809e-654d9badb16c │ Product Dimension Check     │ 11.37361963190184  │
│ 855df54a-6b6e-4b9e-b595-e9bd19316236 │ Material Feed Jam           │ 13.92941176470588  │
│ 9827a7b1-16df-4eb5-b1ab-8f3d77d031a7 │ Die Head Cleaning           │ 44.62602739726027  │
│ a759c756-9ed0-4a00-8c9f-79f3b4bf4c0a │ Temperature Zone Deviation  │ 26.620754716981132 │
│ b606124a-58b9-4fa2-943a-5c2f6482b1ed │ Die Lip Cleaning            │ 28.514285714285716 │
│ b912146a-7274-4533-9825-214f48b41a6f │ Bearing Temperature Alert   │ 85.3               │
│ c1f0d60c-27ef-4c46-8c07-6977c0123e23 │ Hopper Level Warning        │ 8.80062893081761   │
│ d0f0c31b-9368-4155-9ad9-b8a9f4755b88 │ Vacuum System Maintenance   │ 60.08              │
│ dfc5fa17-0fda-4bd6-9258-2d93df449f8f │ Material Changeover         │ 127.41904761904763 │
│ f1024d23-09ec-46e6-8cbd-e026c6c19294 │ Material Contamination      │ 110.43055555555556 │
└──────────────────────────────────────┴─────────────────────────────┴────────────────────┘

¡Hasta luego!